# Tutorial for the FabSim3 Autovalidator: An example in migration prediction

[FabSim3](https://fabsim3.readthedocs.io/en/latest/) is a Python-based automation toolkit for scientific simulation and data processing workflows, licensed under the BSD 3-clause license. It is a component of the [VECMA Toolkit](http://www.vecma-toolkit.eu), which is developed as a part of the [VECMA](http://www.vecma.eu) project funded by the European Union Horizon 2020 research and innovation programme.

FabSim3 supports the use of simple one-liner commands to:

- Organize input, output and environment information, creating a consistent log and making it possible by default to repeat/reproduce runs.
- Establish and run coupled models using the workflow automation functionalities.
- Perform large ensemble simulations (or replicated ones) using a one-line command.
- Enable the execution of simulation and analysis tasks on supercomputers.

Users can perform complex remote tasks from a local command-line, and run single jobs, ensembles of multiple jobs, and dynamic workflows through schedulers such as SLURM, Portable Batch System Professional (PBSPro), LoadLeveller and Quality in Cloud and Grid [(QCG)](http://www.qoscosgrid.org/trac/qcg). FabSim3 stores machine-specific configurations in the repository, and applies it to all applications run on that machine. These configurations are updated by any contributor who feels that a fix or improvement is required.

- #### <p> <span style="color:red"><em> <b> NOTE </b> This setup targets VECMA PSNC Jupyter Notebook platform, if you want to use this tutorial on your local Linux machine, please use FabSim3_Flee_Autovalidator_Linux.ipynb file</em> text</span>.</p>

## <span style="color:#800000"><b> Intalling FabSim3 <b></span>

To install [FabSim3](https://github.com/djgroen/FabSim3), clone the repository using:

In [ ]:
%%bash

if [ ! -d "FabSim3" ]
then
    git clone https://github.com/djgroen/FabSim3
    echo "FabSim3 rep cloned in -> " $PWD
else
    git -C FabSim3 pull
    echo "updating FabSim3 rep cloned in -> " $PWD
fi

### Setup ssh key

Prior to setting up ssh connection, please insert  
- `.bashrc` file in your home directory with the following `export PATH=/opt/conda/bin:${PATH}` and
- `sshd_config` file in your home directory with the following content


`ListenAddress 127.0.0.1:2222
HostKey /etc/ssh-jovyan/ssh_host_rsa_key
HostKey /etc/ssh-jovyan/ssh_host_dsa_key
AuthorizedKeysFile .ssh/authorized_keys
ChallengeResponseAuthentication no
UsePAM yes
Subsystem   sftp    /usr/lib/openssh/sftp-server
PidFile /etc/ssh-jovyan/sshd.pid
#SyslogFacility AUTH
LogLevel DEBUG
StrictModes no`.

To setup ssh connection for executing jobs on localhost, simply run:

In [ ]:
%%bash

PORT=2222

/usr/sbin/sshd -f /home/jovyan/sshd_config
rm -rf ~/.ssh
mkdir ~/.ssh
ssh-keygen -t rsa -N "" -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod og-wx ~/.ssh/authorized_keys
ssh-keyscan -4 -p $PORT localhost >> ~/.ssh/known_hosts

### Dependencies

FabSim3 relies strongly on the following Python modules:

- [Fabric](http://www.fabfile.org) (shown to work with versions 1.5.3 and 1.10.0), 
- [PyYAML](https://pypi.org/project/PyYAML/) (any version), 
- [ruamel.yaml](https://pypi.org/project/ruamel.yaml/)
- [numpy](https://numpy.org/install/) and
- [cryptography](https://pypi.org/project/cryptography/).

These dependencies are marked down in the `requirements.txt` file, as

`fabric3==1.13.1.post1, 
 pyyaml, 
 pytest, 
 pytest-pep8, 
 ruamel.yaml, 
 numpy, 
 cryptography`
 
To install requitements.txt, simply run the following:

In [ ]:
# Install required Python modules
!pip install -r FabSim3/requirements.txt

### Configure `machine_user.yml` file

To set personal configurations, copy `machines_user_example.yml` as `machines_user.yml` in the `FabSim3/fabsim/deploy` folder and run the following sequence of commands, which will change `localhost:` configurations:


In [ ]:
%%bash

PORT=2222

# Configure machines_user.yml
cp FabSim3/fabsim/deploy/machines_user_example.yml FabSim3/fabsim/deploy/machines_user.yml

# Defines your username from local configuration
sed -i "s/your-username/`whoami`/g" FabSim3/fabsim/deploy/machines_user.yml

# Defines port number for localhost connection
sed -i "s#localhost:#localhost:\n  port: $PORT#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines working directory for configs and results directories
sed -i "s#localhost:#localhost:\n  local_configs: \"$PWD/FabSim3/config_files\"#g" FabSim3/fabsim/deploy/machines_user.yml
sed -i "s#localhost:#localhost:\n  local_results: \"$PWD/FabSim3/results\"#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines home path for execution
sed -i "s#localhost:#localhost:\n  home_path_template: \"$PWD/FabSim3/localhost_exe\"#g" FabSim3/fabsim/deploy/machines_user.yml

To view the changes made in the `machines_user.yml` file, simply run:

In [ ]:
import yaml

machines_user_yml = yaml.load(open("FabSim3/fabsim/deploy/machines_user.yml"), Loader=yaml.FullLoader)
print(yaml.dump({'localhost': machines_user_yml['localhost']}))


### Add FabSim3 to your PATH and PYTHONPATH

To add FabSim3 executable file to system `PATH` and setup `PYTHONPATH`:

In [ ]:
import os
import sys

sys.path.insert(0,  r'%s/FabSim3/fabsim/bin' %(os.getcwd()))
sys.path.insert(0,  r'%s/FabSim3/fabsim' %(os.getcwd()))
os.environ['PATH']=r'%s/FabSim3/fabsim/bin' %(os.getcwd())+os.pathsep+os.environ['PATH']

# Test if fabsim command is availble
!which fabsim

## FabSim3 plugins

FabSim3 contains an integrated test infrastructure, more flexible customisation options using a plugin system and  examples to improve usability. These plugins are available in the `FabSim3/plugins` directory:

In [ ]:
%%bash

cat FabSim3/fabsim/deploy/plugins.yml

### Dummy instance - FabDummy plugin

To demonstrate the use of FabSim3 plugin, install the [FabDummy](https://github.com/djgroen/FabDummy) plugin  by running `install_plugin` command:

In [ ]:
# Install FabDummy
!fabsim localhost install_plugin:FabDummy

#### Running FabDummy test

To test the FabDummy plugin, run a dummy job using:

In [ ]:
%%bash

cd FabSim3

# Execute a dummy test job
fabsim localhost dummy:dummy_test

### Migration Modelling - FabFlee plugin

The [FabFlee](https://github.com/djgroen/FabFlee) plugin  aims to automate the construction, execution and analysis of Flee, which is an agent-based modelling toolkit purpose-built for simulating the movement of individuals across geographical locations. An agent-based Flee simulation code is written in Python. For detailed information, see [Flee](https://flee.readthedocs.io/en/latest/index.html#) documentation.

To install Flee, clone the repository by running:

In [ ]:
%%bash

# Clone the Flee repository
if [ ! -d "flee" ]
then
    git clone -b master https://github.com/djgroen/flee.git
    echo "flee rep cloned in -> " $PWD
else
    git -C flee pull
    echo "updating flee rep cloned in -> " $PWD
fi

#### Installing required python modules for Flee 

To install all required Python packages, simply install:

In [ ]:
!pip install pandas matplotlib numpy mpi4py

#### Installing FabFlee

A FabSim3-based FabFlee plugin provides an environment for researchers and organisations to construct and modify simulations, instantiate and execute multiple runs for different policy decisions, as well as to validate and visualise the obtained results against the existing data.

To install FabFlee, run `install_plugin` function using:

In [ ]:
%%bash

cd FabSim3

# Install FabFlee 
fabsim localhost install_plugin:FabFlee

#### Adding the Flee directory location to your machine_user.yml file

Since FabFlee is a plugin for the Flee simulation code, add the current location of the Flee directory to the `machines_user.yml` configuration file:

In [ ]:
%%bash

PORT=2222

# Configure machines_FabFlee_user.yml
cp FabSim3/plugins/FabFlee/machines_FabFlee_user_example.yml FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

In [ ]:
!sed -i  "s#<PATH_TO_FLEE>#$PWD/flee#g" FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

To view changes in `machines_user.yml`, simply run the following

In [ ]:
machines_user_yml = yaml.load(open("FabSim3/plugins/FabFlee/machines_FabFlee_user.yml"), Loader=yaml.FullLoader)

print(yaml.dump({'localhost': machines_user_yml['localhost']}))


#### Executing a migration conflict scenario

To execute FabFlee job on a localhost, execute one of the conflict instances using `flee` function in `FabSim3/plugins/FabFlee/config_files`, such as Mali (mali, simulation period of 300 days), Burundi (burundi, simulation period of 396 days), South Sudan (ssudan, simulation period of 604 days) and Central African Republic (car, simulation period of 820 days). 

For illustration purposes, the conflict scenario of Mali is executed to obtain the simulation output for a simulation period of 50 days, which can be changed up to 300 days.

In [ ]:
%%bash

cd FabSim3

fabsim localhost flee:mali,simulation_period=50

#### Fetching the simulation results to `FabSim3/results` directory

After the job has finished, a message will be printed indicating where the output data resides. To fetch and copy the obtained results to `FabSim3/results` directory, simply execute:

In [ ]:
%%bash

cd FabSim3

fabsim localhost fetch_results

#### Plotting the simulation output

After fetching the simulation results, analyse and interpret simulation output by plotting graphs for individual camps comparing simulation output and UNHCR data, as well as average relative difference graph for the simulated conflict situation.

In [ ]:
%%bash

cd FabSim3

fabsim localhost plot_output:mali_localhost_16,out

To list generated plots, run the following command:

In [ ]:
%%bash

cd FabSim3/results/mali_localhost_16
ls

To view generated plots, simply run:

In [ ]:
from IPython.display import Image 
Image('FabSim3/results/mali_localhost_16/out/camps/Abala.png')

In [ ]:
from IPython.display import Image 
Image('FabSim3/results/mali_localhost_16/out/errors/error.png')

## Autovalidator

There are several simulation instances including different conflict countries and simulation periods in `FabFlee/config_files`. These instances can be executed, analysed and validated using FabFlee (as shown in the previous steps). Importantly, multiple conflict scenarios can be also executed at once for validation purposes. To perform autovalidation on constructed conflict models, simply go to `FabFlee/config_files/validation/SWEEP` directory and list existing conflicts:

In [ ]:
%%bash

cd FabSim3/plugins/FabFlee/config_files/validation/SWEEP

ls

Each conflict scenario contains `input_csv` (input data), `source_data` (data for comparing simulation results and UNHCR historical data) and `run.py`/`run_par.py` execution scripts. 

In [ ]:
%%bash

cd FabSim3/plugins/FabFlee/config_files/validation/SWEEP/burundi2015

ls

Since these simulation instances are sizeable to execute, we scale down the number of agents by factor of 100. Specifically, `simsetting.csv` file allows to perform this:

In [ ]:
%%bash

cat FabSim3/plugins/FabFlee/config_files/validation/simsetting.csv

To execute these conflict scenarios at once, simply run the `validate_flee` function as follows

In [ ]:
%%bash

cd FabSim3

fabsim localhost validate_flee

After simulations are completed, validation values of these conflict instances are calculated and printed in the output. Alternatively, the command below can be executed after completion of simulations to observe the validation outputs for each conflict instance.

In [ ]:
%%bash

cd FabSim3

fabsim localhost validate_flee_output:validation_localhost_4